In [9]:
import pymysql as m
import pandas as pd
from datetime import datetime
from tqdm import notebook
from glob import glob
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import pymysql
import pandas as pd

In [10]:
press_list = ['경향신문', '국민일보', '뉴스1', '뉴시스', '동아일보', '문화일보', '서울신문', '세계일보', '연합뉴스',  '조선일보', '중앙일보', '한겨레', '한국일보']

stop_content = ["무단전재", "재배포금지","저작권자 ⓒ 서울신문사","무단복제 및 전재","무단 전재 및 재배포","제보는 카톡", "☞", "무단 전재-재배포", "▶연합뉴스 앱 지금 바로 다운받기~"]

# rep_list = [u'\xa0', u'\u2027', u'\u30fb', u'\u2024', u'\uff65', u'\u2014', u'\u22c5', u'\u207a', u'\u2219', u'\u2e31', u'\u200d']
rep_list = ['기사내용 요약']

stop_title = ['[부고]', '[인사]', '[표]', '[모멘트]', '[궂긴소식]']

In [11]:
import time
import traceback
def ErrorLog_scraping(error: str):
    current_time = time.strftime("%Y.%m.%d/%H:%M:%S", time.localtime(time.time()))
    with open("error_Log_scraping.txt", "a") as f:
        f.write(f"[{current_time}] - {error}\n")

In [ ]:
def ErrorLog_sql(error: str):
    current_time = time.strftime("%Y.%m.%d/%H:%M:%S", time.localtime(time.time()))
    with open("error_Log_sql.txt", "a") as f:
        f.write(f"[{current_time}] - {error}\n")

In [12]:
def create_soup(url):
    i = 0
    ## 요청 오류시 10번  재시도
    while i < 10 :
        try:
            res = requests.get(url)
            res.raise_for_status()
            soup = BeautifulSoup(res.content, 'html.parser', from_encoding='cp949')
            break
        except:
            i += 1
            if i == 10 :
                err = traceback.format_exc().split('\n')[-2]
                ErrorLog_scraping(str(err))
                return false
                
            
    return soup

In [13]:
def content_scraper(link):
    article_soup  = create_soup(link)
    
    if not article_soup:
        return False
    
    # 기자없는 기사 걸러내기
    journalist = article_soup.select('div>span')[0].text.split('기자')[0].strip() + ' 기자'
    if '입력' in journalist or journalist == ' 기자':
        return False
    
    # 본문 정리
    article = article_soup.find_all('section')[1]
    content = article.find_all(True, attrs={'dmcf-ptype':'general'})
    rst = []
    for para in content:
        for tmp in para.text.split('\n'):
            if tmp.strip() != '':
                rst.append(tmp.strip())
    content = []
    for c in rst:
        for i in stop_content:
            if i in c:
                break
        else:
            for rep in rep_list:
                c = c.replace(rep, '')
            content.append(c)
    content = ' '.join(content)
    if len(content) < 200:
        return False
    return content, journalist

In [14]:
def get_articles(date, category):
    
#     sql용 
    con = m.connect(host = "", user = 'KH', db = 'mintsaveworld' , password = "", charset = "utf8mb4")
    cur = con.cursor()
    sql = """insert into test values (%s,%s,%s,%s,%s,%s,%s,%s); """
    
    
    # 마지막 리스트 페이지 체크
    url = f'https://news.daum.net/breakingnews/{category}?page=999&regDate={date}'
    soup = create_soup(url)
    
    
    if not soup:
        return


    
    last_page = soup.find("em",attrs ={"class": "num_page"})
    last_page_num = int(re.sub(r'[^0-9]', '', last_page.text))
    # last_page_num = 10
    
    for page in notebook.tqdm(range(1, last_page_num+1)):
        url = f'https://news.daum.net/breakingnews/{category}?page={page}&regDate={date}'
        soup = create_soup(url)
        
        if not soup:
#           n페이지 오류 시 그냥 진행
            continue
        
            
        
        news_list = soup.find_all('strong', attrs={'class':'tit_thumb'})
        
        # 리스트의 뉴스 하나씩 처리
        for i in range(len(news_list)-4):
            press = news_list[i].find('span').text.split('·')[0].strip()
            if press not in press_list:
                continue
            title = news_list[i].find('a').text
            tflag = True
            for t in stop_title:
                if t in title:
                    tflag = False
                    break
            if not tflag:
                continue
            link = news_list[i].find('a')['href']
            # print(link, end='\r')
            date = news_list[i].find('a')['href'].split('/')[-1][:8]
            arcid = link.split('/')[-1]
            category = url.split('/')[-1].split('?')[0]
            checker = content_scraper(link)
            
            if not checker:
                
                continue
                
            content, journalist = checker
                
            
            
            q_arcid = arcid
            q_press = press
            q_title = title
            q_content = content
            q_journalist = journalist
            q_date = date
            q_link = link
            q_category = category
            
            
            try:
                cur.execute(sql,(q_arcid, q_press, q_title, q_content, q_journalist, q_date, q_link, q_category))
            except:
                err = traceback.format_exc(chain=False).split('\n')[-2]
                ErrorLog_scraping(str(err))
                

            

    
    con.commit()
    con.close()

In [15]:
from datetime import date
from dateutil.rrule import rrule, DAILY
start_date = date(2020, 6, 24)
end_date = date(2020, 6, 24)


In [16]:
get_articles(start_date, 'society')

  0%|          | 0/1 [00:00<?, ?it/s]